In [3]:
from helper_dask import launch_dask_cluster
import runhouse as rh 

num_nodes = 3
cluster_name = f"py-{num_nodes}"

cluster = rh.cluster(
    name=cluster_name,
    instance_type="r5d.xlarge",
    num_instances=num_nodes,
    provider="aws",
).up_if_not()

launch_dask_cluster(cluster)

try:
    while True:
        pass
except KeyboardInterrupt:
    print("Tore down Dask cluster")

Output()

[LightGBM] [Info] Trying to bind port 38667...
[LightGBM] [Info] Binding port 38667 succeeded
[LightGBM] [Info] Listening...
[LightGBM] [Warning] Connecting to rank 2 failed, waiting for 200 milliseconds
[LightGBM] [Info] Trying to bind port 43435...
[LightGBM] [Info] Binding port 43435 succeeded
[LightGBM] [Info] Listening...


[LightGBM] [Info] Connected to rank 0
[LightGBM] [Info] Connected to rank 0
[LightGBM] [Info] Connected to rank 1
[LightGBM] [Info] Connected to rank 1
[LightGBM] [Info] Connected to rank 2
[LightGBM] [Info] Local rank: 2, total number of machines: 3
[LightGBM] [Info] Connected to rank 2
[LightGBM] [Info] Local rank: 1, total number of machines: 3
[LightGBM] [Info] Local rank: 0, total number of machines: 3


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.084543 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 586
[LightGBM] [Info] Number of data points in the train set: 908382, number of used features: 7
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.078712 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 586
[LightGBM] [Info] Number of data points in the train set: 1617541, number of used features: 7


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.143156 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 586
[LightGBM] [Info] Number of data points in the train set: 1435617, number of used features: 7


[LightGBM] [Info] Start training from score 3.271060
[LightGBM] [Info] Start training from score 3.271060
[LightGBM] [Info] Start training from score 3.271060


INFO | 2024-11-06 22:49:17 | runhouse.resources.hardware.cluster:1745 | Running command on py-3: echo "hello"
INFO | 2024-11-06 22:49:17 | runhouse.resources.hardware.ssh_tunnel:238 | SSH tunnel on to server's port 32300 via server's ssh port 22 already created with the cluster.
INFO | 2024-11-06 22:49:18 | runhouse.servers.http.http_client:439 | Calling _cluster_default_env._run_command


INFO:httpx:HTTP Request: GET http://127.0.0.1:32301/logs/_cluster_default_env__run_command@20241106@224918_008325/pickle "HTTP/1.1 200 OK"


INFO | 2024-11-06 22:49:19 | runhouse.servers.http.http_client:503 | Time to call _cluster_default_env._run_command: 1.55 seconds
INFO | 2024-11-06 22:49:20 | runhouse.resources.hardware.cluster:1745 | Running command on py-3: pip install "dask[distributed]"
INFO | 2024-11-06 22:49:24 | runhouse.resources.hardware.cluster:1745 | Running command on py-3: dask worker tcp://172.31.89.245:8786
2024-11-07 03:49:25,770 - distributed.nanny - INFO -         Start Nanny at: 'tcp://172.31.89.245:36765'
2024-11-07 03:49:26,512 - distributed.worker - INFO -       Start worker at:  tcp://172.31.89.245:44593
2024-11-07 03:49:26,512 - distributed.worker - INFO -          Listening to:  tcp://172.31.89.245:44593
2024-11-07 03:49:26,512 - distributed.worker - INFO -          dashboard at:        172.31.89.245:40777
2024-11-07 03:49:26,512 - distributed.worker - INFO - Waiting to connect to:   tcp://172.31.89.245:8786
2024-11-07 03:49:26,512 - distributed.worker - INFO - --------------------------------